# Vector search in Python (Azure AI Search)
이 노트북 예제에서는 Azure AI Search의 Python SDK를 사용해서 다음과 같은 검색 쿼리를 만들 수 있다.

- 키워드 검색
- 벡터 검색
- 하이브리드 검색
- 의미 체계 하이브리드 검색



# 사전 준비
이 파이썬 예제를 실행하려면 다음과 같은 환경이 필요하다:
- Azure AI Search 리소스의 엔드포인트 및 관리자 API 키
- Azure OpenAI Service에 접근할 수 있는 승인된 Azure 구독
- Azure OpenAI Service에 배포된 `text-embedding-ada-002` Embeddings 모델. 이 모델이 사용하는 API 버전은 `2024-02-01`다. 배포 이름은 모델 이름과 동일하게 `text-embedding-ada-002`를 사용한다.
- Azure OpenAI Service 연동 및 모델 정보
  - OpenAI API 키
  - OpenAI Embeddings 모델의 배포 이름
  - OpenAI API 버전
- Python(이 예제는 버전 3.12.4로 테스트 했다.)

이 예제에서는 Visual Studio Code와 [Jupyter extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter)를 사용한다.

## 패키지 설치

In [ ]:
!pip install azure-search-documents==11.4.0
!pip install openai==1.35.3

## 라이브러리 및 환경변수 불러오기

In [ ]:
import azure.search.documents
azure.search.documents.__version__

In [ ]:
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient
from azure.search.documents.models import (
    QueryAnswerType,
    QueryCaptionType,
    QueryType,
    VectorizedQuery,    
)

## 연동 설정

In [ ]:
service_endpoint: str = "<Your search service endpoint>"
service_query_key: str = "<Your search service query key>"
index_name: str = "gptkbindex" # 00_DataIngest_AzureAISearch_PythonSDK.ipynb 에서 생성한 인덱스 이름
model: str = "text-embedding-ada-002"

credential = AzureKeyCredential(service_query_key)

# 1. 키워드 검색
가장 간단한 검색인 키워드 검색부터 살펴보자. `ko.lucene`이라는 표준 한국어 분석기에 탑재된 사전 기반 토크나이저를 사용해서 토큰으로 분할한다. 분할된 토큰으로 역인덱스(역색인)를 생성한다.

그리고 TF/IDF 기반의 [BM25](https://en.wikipedia.org/wiki/Okapi_BM25) 채점 알고리즘을 사용해서 문장 내 토큰의 일치빈도를 보며 연관성 점수를 계산한다. 내부적으로는 토큰의 희소성과 문장의 밀도 등에도 가중치를 부여한다. 키워드 검색에서는 오타가 생기면 가령 아래의 예제처럼 **몽골**의 오타인 '몽돌'로 검색하면 기대한 결과가 나타나지 않거나 더 적게 나타날 수 있다.

In [ ]:
query = "몽돌 전쟁을 대비하기 위해 어떤 준비를 했나요?"  
search_client = SearchClient(service_endpoint, index_name, credential=credential)
docs = search_client.search(
    search_text=query,
    top=3,
    highlight_fields="content-3",
    select="sourcepage,content,category"
) 

for doc in docs:
    print(f"Source: {doc['sourcepage']}")  
    print(f"Score: {doc['@search.score']}")  
    print(f"Content: {doc['content']}")  
    print(f"Category: {doc['category']}\n")

# 2. 벡터 유사도 검색

## Azure OpenAI 설정

In [ ]:
AZURE_OPENAI_API_KEY = "Your OpenAI API Key"
AZURE_OPENAI_ENDPOINT = "https://<Your OpenAI Service>.openai.azure.com/"

In [ ]:
from openai import AzureOpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt  

client = AzureOpenAI(
  api_key = AZURE_OPENAI_API_KEY,  
  api_version = "2024-02-01",
  azure_endpoint = AZURE_OPENAI_ENDPOINT
)

@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
# title 필드와 content 필드의 벡터를 생성하는 함수
def generate_embeddings(text, model=model):
    return client.embeddings.create(input = [text], model=model).data[0].embedding

# 2.1. 간단한 벡터 검색
'몽**돌** 전쟁을 대비하기 위해 어떤 준비를 했나요?'라고 일부러 철자를 틀리게 입력한 쿼리로 검색을 해보자.
`text-embeddings-ada-002`로 생성한 벡터를 검색하면 일치하는 키워드를 찾는데 얽매이지 않고 문장의 유사도만을 고려해서 검색한다. 오타도 적절하게 무시하면서 검색하기 때문에 키워드 검색보다 더 나은 결과를 보여준다.

In [ ]:
query = "몽돌 전쟁을 대비하기 위해 어떤 준비를 했나요?"  

search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="embedding")

docs = search_client.search(
    search_text=None,
    vector_queries= [vector_query],
    select=["sourcepage", "content", "category"],
)

for doc in docs:
    print(f"Source: {doc['sourcepage']}")  
    print(f"Score: {doc['@search.score']}")  
    print(f"Content: {doc['content']}")  
    print(f"Category: {doc['category']}\n")

## 2.1.1. 다국어 처리
`text-embeddings-ada-002`의 다국어 처리 능력을 확인하기 위해 한국어 문서를 영어로 검색해보자.

In [ ]:
query = "What preparations were made to prepare for the Mongol invasion?"  

search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="embedding")

docs = search_client.search(
    search_text=None,
    vector_queries= [vector_query],
    select=["sourcepage", "content", "category"],
)

for doc in docs:
    print(f"Source: {doc['sourcepage']}")  
    print(f"Score: {doc['@search.score']}")  
    print(f"Content: {doc['content']}")  
    print(f"Category: {doc['category']}\n")

# 2.2. 하이브리드 검색
하이브리드 검색은 키워드 검색과 벡터 검색 모두를 쿼리에 사용한다. 키워드 검색 스코어를 구할 떄는 Okapi BM25 알고리즘을 사용해서 스코어를 계산하고, 벡터 검색에는 코사인 유사도를 기준으로 스코어를 계산한다. 이 서로 다른 계산 결과를 융합하는 방법으로는 [Reciprocal Rank Fusion(RRF)](https://learn.microsoft.com/azure/search/hybrid-search-ranking)을 사용한다. RRF는 두 방식으로 계산한 문서 랭크의 역수의 합을 구해서 문서 순위를 기준으로 어느 쪽의 랭크라도 상위에 있는 쪽이 스코어가 높아지는 구조다.

In [ ]:
query = "최씨정권을 타도하고 최고 권력자가 된 인물은?"  

search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=50, fields="embedding")

docs = search_client.search(
    search_text=query,
    vector_queries= [vector_query],
    select=["sourcepage", "content", "category"],
    top=3
)

for doc in docs:
    print(f"Source: {doc['sourcepage']}")  
    print(f"Score: {doc['@search.score']}")  
    print(f"Content: {doc['content']}")  
    print(f"Category: {doc['category']}\n")

# 2.3 의미 체계 하이브리드 검색
의미 체계 하이브리드 검색(하이브리드 검색 + 의미 체계 순위 지정)은 Azure AI Search에서만 지원하는 검색 기능으로, 하이브리드 검색과 검색 결과를 정확도가 높은 순서로 정렬하는 순위 재책정 기능(의미 체계 순위 지정)을 조합한 고도화된 검색 방법이다. 순위 재책정에는 마이크로소프트가 만든 언어 모델인 [Turing](https://techcommunity.microsoft.com/t5/ai-azure-ai-services-blog/introducing-multilingual-support-for-semantic-search-on-azure/ba-p/2385110) 모델을 사용한다.

In [ ]:
query = "무신정변을 일으켜 무신정권을 수립한 무신 3명은?"  

search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=50, fields="embedding")

docs = search_client.search(
    search_text=query,
    vector_queries= [vector_query],
    select=["sourcepage", "content", "category"],
    query_type=QueryType.SEMANTIC, 
    semantic_configuration_name='default', 
    query_caption=QueryCaptionType.EXTRACTIVE, 
    query_answer=QueryAnswerType.EXTRACTIVE,
    top=3
)

semantic_answers = docs.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")
    
for doc in docs:
    print(f"Source: {doc['sourcepage']}")  
    print(f"Score: {doc['@search.score']}")  
    print(f"Content: {doc['content']}")  
    print(f"Category: {doc['category']}\n")
    
    captions = doc["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")

## 2.3.1. 의미 체계 답변 및 의미 체계 캡션
**의미 체계 답변**은 검색 문서에서 가장 가까워보이는 구절을 추출해서 표시하는 응답이다. 구절 중에서도 더 답변에 가까운 부분에는 자동적으로 강조 표시가 추가된다. 의미 체계 답변을 반환받으려면 답변의 언어적 특성을 가진 구절이나 문장이 검색 문서에 존재해야 하며, 검색 쿼리 자체가 질문이어야 한다. 모델은 사용할 수 있는 콘텐츠에서 일련의 답변 후보들을 추출하고 **충분히 높은 신뢰수준에 도달할 때만** 의미 체계 답변을 제안한다.

**의미 체계 캡션**은 정확한 의미 체계 답변을 얻지 못한 경우에도 관련성 높은 검색 결과를 일부분 추출(추출적 요약)할 수 있다. 의미 체계 캡션은 새로운 문장이나 구절을 생성하는 작업이 아니므로 신뢰성이 필요한 설명이나 정의를 표시하기 위해 사용하는 것이 적절하다.

In [ ]:
query = "최충헌이 쿠데타를 일으킨 시기는 언제인가?"  

search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=50, fields="embedding")

docs = search_client.search(
    search_text=query,
    vector_queries= [vector_query],
    select=["sourcepage", "content", "category"],
    query_type=QueryType.SEMANTIC, 
    semantic_configuration_name='default', 
    query_caption=QueryCaptionType.EXTRACTIVE, 
    query_answer=QueryAnswerType.EXTRACTIVE,
    top=3
)

semantic_answers = docs.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")
    
for doc in docs:
    captions = doc["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")

In [ ]:
query = "몽골 전쟁을 대비하며 간행한 경전은 무엇인가?"

search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=50, fields="embedding")

docs = search_client.search(
    search_text=query,
    vector_queries= [vector_query],
    select=["sourcepage", "content", "category"],
    query_type=QueryType.SEMANTIC, 
    semantic_configuration_name='default', 
    query_caption=QueryCaptionType.EXTRACTIVE, 
    query_answer=QueryAnswerType.EXTRACTIVE,
    top=3
)

semantic_answers = docs.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")
    
for doc in docs:
    captions = doc["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")